## Import libs

In [1]:
import os
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
sc=SparkContext()
sqlContext=SQLContext(sc)
import urllib
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType ,DoubleType ,BooleanType, IntegerType
from pyspark.sql.functions import udf ,col ,upper, avg
import numpy
import math

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor ,LinearRegression ,GeneralizedLinearRegression
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors 
from pyspark.ml.linalg import VectorUDT

from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors


from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor ,LinearRegression ,GeneralizedLinearRegression


from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#from pyspark.mllib.evaluation import RegressionMetrics


from pyspark.ml.linalg import Vectors 
from pyspark.ml.linalg import VectorUDT

## Read input

In [3]:
numerai_train_df     = sqlContext.read.option("header","true").csv('file:///home/garth/Downloads/numerai20161126/numerai_training_data.csv',inferSchema='true')
numerai_tournament_df= sqlContext.read.option("header","true").csv('file:///home/garth/Downloads/numerai20161126/numerai_tournament_data.csv',inferSchema='true')

print "length of training set:"
print (numerai_train_df.count())
print "length of tournament set:"
print (numerai_tournament_df.count())

length of training set:
96320
length of tournament set:
135270


## Convert input for use in ml lib algos

In [4]:
train_df=numerai_train_df.rdd.map(lambda x: (float(x.target),Vectors.dense([
                x.feature1
                ,x.feature2
                ,x.feature3
                ,x.feature4
                ,x.feature5
                ,x.feature6
                ,x.feature7
                ,x.feature8
                ,x.feature9
                ,x.feature10
                ,x.feature11
                ,x.feature12
                ,x.feature13
                ,x.feature14
                ,x.feature15
                ,x.feature16
                ,x.feature17
                ,x.feature18
                ,x.feature19
                ,x.feature20
                ,x.feature21
                ]))).toDF(["label","features"])

tournament_df=numerai_tournament_df.rdd.map(lambda x: (x.t_id,Vectors.dense([
                x.feature1
                ,x.feature2
                ,x.feature3
                ,x.feature4
                ,x.feature5
                ,x.feature6
                ,x.feature7
                ,x.feature8
                ,x.feature9
                ,x.feature10
                ,x.feature11
                ,x.feature12
                ,x.feature13
                ,x.feature14
                ,x.feature15
                ,x.feature16
                ,x.feature17
                ,x.feature18
                ,x.feature19
                ,x.feature20
                ,x.feature21
                ]))).toDF(["t_id","features"])



## Explore data

In [5]:
#print "success rate of training set:"
#print str(float(train_df.where("label='1'").count())/train_df.count())

success rate of training set:
0.505170265781


## Split Data for training and testing

In [6]:
(trainingData, testData) = train_df.randomSplit([0.7, 0.3],1234)

## Define log loss function

In [7]:
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

## PCA

In [8]:
pca = PCA(k=21, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(trainingData)
trainingDataPCA = model.transform(trainingData).drop("features")
trainingDataPCA = trainingDataPCA.withColumn("features",trainingDataPCA.pcaFeatures).drop("pcaFeatures")

testingDataPCA = model.transform(testData).drop("features")
testingDataPCA = testingDataPCA.withColumn("features",testingDataPCA.pcaFeatures).drop("pcaFeatures")

#testingDataPCA.show(2,False)
#trainingDataPCA.show(1)

## Perform Random Forest Regression on PCA features

In [9]:

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(trainingDataPCA)

rf = RandomForestRegressor(featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[featureIndexer, rf])
model = pipeline.fit(trainingDataPCA)
predictions_rfr = model.transform(testingDataPCA)
training_rfr=model.transform(trainingDataPCA)
#predictions_rfr.show(5)


In [10]:
#print "true  + :" + str( predictions_rfr.where("label=='1.0' AND prediction > 0.5").count() )
#print "false - :" + str( predictions_rfr.where("label=='1.0' AND prediction < 0.5").count() )
#print "false + :" + str( predictions_rfr.where("label=='0.0' AND prediction > 0.5").count() )
#print "true  - :" + str( predictions_rfr.where("label=='0.0' AND prediction < 0.5").count() )
#print "uncertain:" + str (predictions_rfr.where("prediction=='0.5'").count())
#print ("total count:" + str(predictions_rfr.count()))
#print ""

print "accuracy= "+ str(float(training_rfr.where("(label=='1.0' AND prediction > 0.5) OR(label=='0.0' AND prediction < 0.5)").count())/training_rfr.count()) +"%"
print "percentage of 1s label :  "+ str(float(training_rfr.where("label==1.00").count())/training_rfr.count()) +"%"
print "percentage of 1s predxn:  "+ str(float(training_rfr.where("prediction>'.5'").count())/training_rfr.count()) +"%"
#training_rfr.describe().show()
#print "describe incorrect: "
#training_rfr.where("(label==0 and prediction>0.5) or (label==1 and prediction<0.5)").describe().show()
#print "describe correct: "
#training_rfr.where("(label==1 and prediction>0.5) or (label==0 and prediction<0.5)").describe().show()

act=[float(i.label) for i in training_rfr.collect()]
pred=[float(i.prediction) for i in training_rfr.collect()]
print "logloss:" 
print logloss(act,pred)


print "__________________________________________________________________________________"

print "accuracy= "+ str(float(predictions_rfr.where("(label=='1.0' AND prediction > 0.5) OR(label=='0.0' AND prediction < 0.5)").count())/predictions_rfr.count()) +"%"
print "percentage of 1s label :  "+ str(float(predictions_rfr.where("label==1.00").count())/predictions_rfr.count()) +"%"
print "percentage of 1s predxn:  "+ str(float(predictions_rfr.where("prediction>'.5'").count())/predictions_rfr.count()) +"%"
#predictions_rfr.describe().show()
#print "describe incorrect: "
#predictions_rfr.where("(label==0 and prediction>0.5) or (label==1 and prediction<0.5)").describe().show()
#print "describe correct: "
#predictions_rfr.where("(label==1 and prediction>0.5) or (label==0 and prediction<0.5)").describe().show()

act=[float(i.label) for i in predictions_rfr.collect()]
pred=[float(i.prediction) for i in predictions_rfr.collect()]
print "logloss:" 
print logloss(act,pred)


accuracy= 0.549694836727%
percentage of 1s label :  0.503125881707%
percentage of 1s predxn:  0.590323874014%
logloss:
0.68898841141
__________________________________________________________________________________
accuracy= 0.518375375272%
percentage of 1s label :  0.509920977259%
percentage of 1s predxn:  0.594706511612%
logloss:
0.691933340851


In [11]:
print "percentage of very low in correct:  "+ str(100*float(predictions_rfr.where("prediction<'.4' and label=0").count())/predictions_rfr.count()) +"%"
print "percentage of very low in incorrect:  "+ str(100*float(predictions_rfr.where("prediction<'.4' and label=1").count())/predictions_rfr.count()) +"%"

def improve(x):
    if x < 0.47:
        return (0.47)
    else: return x
    
improve_answer_udf=udf(lambda x: improve(x), DoubleType())

predictions_rfr=predictions_rfr.withColumn("prediction2",improve_answer_udf(predictions_rfr.prediction)).drop("prediction")
predictions_rfr=predictions_rfr.withColumn("prediction",predictions_rfr.prediction2).drop("prediction2")
act=[float(i.label) for i in predictions_rfr.collect()]
pred=[float(i.prediction) for i in predictions_rfr.collect()]
print "logloss:" 
print logloss(act,pred)

percentage of very low in correct:  0.0%
percentage of very low in incorrect:  0.0%
logloss:
0.691974302425


In [12]:
#Exploring answer:
#predictions_rfr.show(2)
print str(float(predictions_rfr.where("prediction<0.50").count())/predictions_rfr.count())
print str(float(predictions_rfr.where("label=0").count())/predictions_rfr.count())
print str(float(predictions_rfr.where("prediction<0.50 and label=0 OR (prediction>0.5 and label=1)").count())/predictions_rfr.count())

0.405293488388
0.490079022741
0.518375375272


## Perform Generalised Linear Regression on PCA features

In [13]:
glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=1000, regParam=0.3)
model = glr.fit(trainingDataPCA)
predictions_glr = model.transform(testingDataPCA)
predictions_glr=predictions_glr.withColumnRenamed("prediction","probability")#.drop("features")
#predictions_glr.show(2)

In [14]:
#print "true  + :" + str( predictions_glr.where("label=='1.0' AND prediction > 0.5").count() )
#print "false - :" + str( predictions_glr.where("label=='1.0' AND prediction < 0.5").count() )
#print "false + :" + str( predictions_glr.where("label=='0.0' AND prediction > 0.5").count() )
#print "true  - :" + str( predictions_glr.where("label=='0.0' AND prediction < 0.5").count() )
#print "uncertain:" + str (predictions_glr.where("prediction=='0.5'").count())
#print ("total count:" + str(predictions_glr.count()))
print ""
print "accuracy= "+ str(float(predictions_glr.where("(label=='1.0' AND prediction > 0.5) OR(label=='0.0' AND prediction < 0.5)").count())/predictions_glr.count()) +"%"

act=[float(i.label) for i in predictions_glr.collect()]
pred=[float(i.probability) for i in predictions_glr.collect()]
print "logloss:" 
print logloss(act,pred)


accuracy= 0.524690292971%
logloss:
0.691567438542


## XGBT

In [15]:
gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=20)
pipeline = Pipeline(stages=[featureIndexer, gbt])
model = pipeline.fit(trainingDataPCA)
predictions_xgbtr = model.transform(testingDataPCA)

print "accuracy= "+ str(float(100*predictions_xgbtr.where("(label=='1.0' AND prediction > 0.5) OR(label=='0.0' AND prediction < 0.5)").count())/predictions_xgbtr.count()) +"%"
act=[float(i.label) for i in predictions_xgbtr.collect()]
pred=[float(i.prediction) for i in predictions_xgbtr.collect()]
print "logloss:" 
print logloss(act,pred)
predictions_xgbtr.show(1)

accuracy= 52.137754926%
logloss:
0.695770143827
+-----+--------------------+--------------------+------------------+
|label|            features|     indexedFeatures|        prediction|
+-----+--------------------+--------------------+------------------+
|  0.0|[-0.8712915406536...|[-0.8712915406536...|0.5180750502233212|
+-----+--------------------+--------------------+------------------+
only showing top 1 row



## MLPC

In [17]:
print "features length"
print (len(trainingDataPCA.first()['features']))
layers = [21,25,2]
trainer = MultilayerPerceptronClassifier(maxIter=2000, layers=layers, blockSize=128, seed=15)
model = trainer.fit(trainingDataPCA)

# compute accuracy on the test set
predictions_MLPC = model.transform(testingDataPCA)
print predictions_MLPC.count()
predictionAndLabels = predictions_MLPC.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy: " + str(evaluator.evaluate(predictionAndLabels)))


features length
21
28979
Accuracy: 0.524966354947


In [18]:
print predictions_MLPC.where("prediction=1").count()
print predictions_MLPC.where("prediction=0").count()

17561
11418


In [19]:
print "percentage of 1 labels: "+str(100*float(testingDataPCA.where("label==1").count())/testingDataPCA.count())

percentage of 1 labels: 50.9920977259


## Join tables

In [20]:
predictions_rfr.createOrReplaceTempView("predictions_rfr")
predictions_glr.createOrReplaceTempView("predictions_glr")
predictions_MLPC.createOrReplaceTempView("predictions_MLPC")


In [36]:
sqlContext.sql(
            "select * \
                from predictions_rfr join predictions_glr \
                on predictions_rfr.features = predictions_glr.features \
                inner join predictions_MLPC \
                on predictions_MLPC.features = predictions_rfr.features limit 2").show()

+-----+--------------------+--------------------+-------------------+-----+--------------------+-------------------+-----+--------------------+----------+
|label|            features|     indexedFeatures|         prediction|label|            features|        probability|label|            features|prediction|
+-----+--------------------+--------------------+-------------------+-----+--------------------+-------------------+-----+--------------------+----------+
|  1.0|[-1.2685594350497...|[-1.2685594350497...|0.49783679493286714|  1.0|[-1.2685594350497...| 0.5017600762929864|  1.0|[-1.2685594350497...|       1.0|
|  0.0|[-1.1412805205326...|[-1.1412805205326...| 0.5018745107502193|  0.0|[-1.1412805205326...|0.49895845187150023|  0.0|[-1.1412805205326...|       0.0|
+-----+--------------------+--------------------+-------------------+-----+--------------------+-------------------+-----+--------------------+----------+



In [35]:
ensemble_df = sqlContext.sql(
            "select predictions_rfr.features as feature \
            ,predictions_rfr.label as label \
            ,predictions_rfr.prediction as rfr_pred \
            ,predictions_glr.probability as glr_pred \
            ,predictions_MLPC.prediction as mlpc_pred \
                from predictions_rfr join predictions_glr \
                on predictions_rfr.features = predictions_glr.features \
                inner join predictions_MLPC \
                on predictions_MLPC.features = predictions_rfr.features")

ensemble_df.show()

+--------------------+-----+-------------------+-------------------+---------+
|             feature|label|           rfr_pred|           glr_pred|mlpc_pred|
+--------------------+-----+-------------------+-------------------+---------+
|[-1.2685594350497...|  1.0|0.49783679493286714| 0.5017600762929864|      1.0|
|[-1.1412805205326...|  0.0| 0.5018745107502193|0.49895845187150023|      0.0|
|[-0.9796959768020...|  0.0|0.49673980959551906| 0.5033670463598825|      0.0|
|[-0.7917442126579...|  0.0| 0.4907994115811835| 0.5091977003021579|      1.0|
|[-0.7444916352099...|  1.0| 0.5226694863003838| 0.5140961628993246|      1.0|
|[-0.7377009368928...|  0.0| 0.5290123633865724| 0.5339846974012757|      1.0|
|[-0.7324081107176...|  1.0| 0.5317641762214111| 0.5151858849838031|      1.0|
|[-0.6038519329776...|  1.0|  0.521207141729467| 0.5152043743996617|      1.0|
|[-0.5839139149905...|  1.0| 0.5024739825811787| 0.5030375076810534|      1.0|
|[-0.5447004925595...|  0.0| 0.4812560321128886| 0.4

In [38]:
print "percentage in agreement rfr and glr: "\
    +str(float(ensemble_df.where("rfr_pred>0.5 and glr_pred>0.5").count())/ensemble_df.count())

percentage in agreement rfr and glr: 0.489595914283


In [39]:
print "percentage in agreement rfr and mlpc: "\
    +str(float(ensemble_df.where("rfr_pred>0.5 and mlpc_pred>0.5").count())/ensemble_df.count())

percentage in agreement rfr and mlpc: 0.494944615066


In [40]:
print "percentage in agreement mlpc and glr: "\
    +str(float(ensemble_df.where("glr_pred>0.5 and mlpc_pred>0.5").count())/ensemble_df.count())

percentage in agreement mlpc and glr: 0.54349701508


In [41]:
print "percentage in agreement mlpc and glr and rfr: "\
    +str(float(ensemble_df.where("glr_pred>0.5 and mlpc_pred>0.5 and rfr_pred>0.5").count())/ensemble_df.count())

percentage in agreement mlpc and glr and rfr: 0.464198212499


In [60]:
#print "avg where they agree: "\
#    +str(float(ensemble_df.where("glr_pred>0.5 and mlpc_pred>0.5 and rfr_pred>0.5")))

#print "label average"
#ensemble_df.agg(avg(col("label"))).show()

#print "glr and rfr average"
#ensemble_df.agg(avg(col("glr_pred"))).show()
#ensemble_df.agg(avg(col("rfr_pred"))).show()

#ensemble_df.where("(glr_pred>0.5 and mlpc_pred>0.5 and rfr_pred>0.5)").agg(avg(col("glr_pred"))).show()
#ensemble_df.where("(glr_pred>0.5 and mlpc_pred>0.5 and rfr_pred>0.5)").agg(avg(col("rfr_pred"))).show()

#ensemble_df.where("glr_pred<0.5 and mlpc_pred<0.5 and rfr_pred<0.5").agg(avg(col("glr_pred"))).show()
#ensemble_df.where("glr_pred<0.5 and mlpc_pred<0.5 and rfr_pred<0.5").agg(avg(col("rfr_pred"))).show()

#print "average where rfr,flr and mlpc models agree"
#ensemble_df.where("(glr_pred>0.5 and mlpc_pred>0.5 and rfr_pred>0.5) or (glr_pred<0.5 and mlpc_pred<0.5 and rfr_pred<0.5)").agg(avg(col("glr_pred"))).show()
#ensemble_df.where("(glr_pred>0.5 and mlpc_pred>0.5 and rfr_pred>0.5) or (glr_pred<0.5 and mlpc_pred<0.5 and rfr_pred<0.5)").agg(avg(col("rfr_pred"))).show()    

models_agree_df=ensemble_df.where("(glr_pred>0.5 and mlpc_pred>0.5 and rfr_pred>0.5) \
                                or (glr_pred<0.5 and mlpc_pred<0.5 and rfr_pred<0.5)")    
models_agree_df.show()
#model_disagree_df=ensemble_df.where("").show()

label average
+------------------+
|        avg(label)|
+------------------+
|0.5099209772593948|
+------------------+

+--------------------+-----+-------------------+------------------+---------+
|             feature|label|           rfr_pred|          glr_pred|mlpc_pred|
+--------------------+-----+-------------------+------------------+---------+
|[-0.7444916352099...|  1.0| 0.5226694863003838|0.5140961628993246|      1.0|
|[-0.7377009368928...|  0.0| 0.5290123633865724|0.5339846974012757|      1.0|
|[-0.7324081107176...|  1.0| 0.5317641762214111|0.5151858849838031|      1.0|
|[-0.6038519329776...|  1.0|  0.521207141729467|0.5152043743996617|      1.0|
|[-0.5839139149905...|  1.0| 0.5024739825811787|0.5030375076810534|      1.0|
|[-0.5447004925595...|  0.0| 0.4812560321128886|0.4922794454450084|      0.0|
|[-0.5402901756806...|  1.0| 0.5255769876237133|0.5102750774864876|      1.0|
|[-0.5330586231326...|  0.0| 0.5083007731690998|0.5034193361511321|      1.0|
|[-0.4950753722882...|

In [66]:


models_agree_df=models_agree_df.withColumn("prediction",(models_agree_df.rfr_pred+models_agree_df.glr_pred)/2)

act=[float(i.label) for i in models_agree_df.collect()]
pred=[float(i.prediction) for i in models_agree_df.collect()]
print "logloss:" 
print logloss(act,pred)
predictions_xgbtr.show(1)

logloss:
0.691125313353
+-----+--------------------+--------------------+------------------+
|label|            features|     indexedFeatures|        prediction|
+-----+--------------------+--------------------+------------------+
|  0.0|[-0.8712915406536...|[-0.8712915406536...|0.5180750502233212|
+-----+--------------------+--------------------+------------------+
only showing top 1 row

